<a href="https://colab.research.google.com/github/vveitch/causality-tutorials/blob/main/ATE_Estimation_with_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ATT Estimation Tutorial

This tutorial gives a short example for how to estimate average treatment effect on the treated using machine learning methods

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os

**Load and Format LaLonde Observational Data**

In [ ]:
def make_data_lalonde(df):
    df_new = df.drop(['nodegree'], axis=1)
    df_new['pos74'] = (df_new['RE74'] > 0).astype(int)
    df_new['pos75'] = (df_new['RE75'] > 0).astype(int)
    df_new['treatment'] = df_new['treatment'].astype(int)
    return df_new


col_names = ['treatment', 'age', 'education', 'black',
             'hispanic', 'married', 'nodegree', 'RE74', 'RE75', 'RE78']
control = pd.read_csv('https://raw.githubusercontent.com/anishazaveri/austen_plots/master/data/imbens-raw/psid_controls.txt', header=None, sep=r"\s\s", names=col_names, engine='python')
treatment = pd.read_csv('https://raw.githubusercontent.com/anishazaveri/austen_plots/master/data/imbens-raw/nswre74_treated.txt', header=None, sep=r"\s\s", names=col_names, engine='python')

lalonde1 = pd.concat([control, treatment]).reset_index(drop=True)
lalonde1 = make_data_lalonde(lalonde1)

In [ ]:
lalonde1.head()

,treatment,age,education,black,hispanic,married,RE74,RE75,RE78,pos74,pos75
0,0,47.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0,50.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0
2,0,44.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,0,28.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0
4,0,54.0,12.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0


**Fit Lalonde**

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
confounders = lalonde1.drop(columns=['RE78', 'treatment'])
outcome = lalonde1['RE78']
treatment = lalonde1['treatment']

For reference, compute the naive point estimate of the effect we'd get if don't control for confounding. In this case, there's a negative association between treatment and the outcome (i.e., job training is associated with reduced income)

In [ ]:
outcome[treatment==1].mean()-outcome[treatment==0].mean()

-15204.777468026874

# Specify Nuisance Function Models

The next step is to specify models for the conditional expected outcome and propensity score

In [ ]:
# specify a model for the conditional expected outcome
Q_model = RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=100, max_depth=None)

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)

test_mse=mean_squared_error(y_pred, y_test)
print(f"Test MSE of fit model {test_mse}") 
baseline_mse=mean_squared_error(y_train.mean()*np.ones_like(y_test), y_test)
print(f"Test MSE of no-covariate model {baseline_mse}")

Test MSE of fit model 107186056.84174497
Test MSE of no-covariate model 246319790.55062827


In [ ]:
# specify a model for the propensity score
g_model = RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100, max_depth=5)

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

test_ce=log_loss(a_test, a_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.0789947383398904
Test CE of no-covariate model 0.21817471356014154


# Use cross fitting to get get predicted outcomes and propensity scores for each unit

In [ ]:
# helper functions to implement the cross fitting

def treatment_k_fold_fit_and_predict(model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: sklearn model, needs to implement fit and predict_prob
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
      X_train = X.loc[train_index]
      A_train = A.loc[train_index]
      model.fit(X_train, A_train)

      # get predictions for split
      predictions[test_index] = model.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions


def outcome_k_fold_fit_and_predict(model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: sklearn model, needs to implement fit and either predict_prob or predict
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
      X_train = X_w_treatment.loc[train_index]
      y_train = y.loc[train_index]
      model.fit(X_train, y_train)

      if output_type =='binary':
        predictions0[test_index] = model.predict_proba(X0.loc[test_index])[:, 1]
        predictions1[test_index] = model.predict_proba(X1.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions0[test_index] = model.predict(X0.loc[test_index])
        predictions1[test_index] = model.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [ ]:
g = treatment_k_fold_fit_and_predict(g_model, X=confounders, A=treatment, n_splits=10)

In [ ]:
Q0,Q1=k_fold_fit_and_predict_Y(Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

In [ ]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.280187,327.275848,2005.302566,0,0.0
1,0.221749,2460.018120,4178.193550,0,0.0
2,0.469797,24.561530,2236.205678,0,0.0
3,0.586396,10551.835256,8920.218158,0,0.0
4,0.015727,0.000000,1633.668328,0,0.0


# Combine predicted values and data into estimate of ATT

In [ ]:
def psi_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATT
  This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
  """

  if prob_t is None:
    prob_t = A.mean() # estimate marginal probability of treatment

  tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
  scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [ ]:
tau_hat, std_hat = psi_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is 1265.3429523217906 pm 1577.8958916776455


In [ ]:
# The LaLonde data has severe overlap issues. Lets try computing the estimate restricted to a population with only reasonable propensity scores
in_overlap_popluation = (0.05 < data_and_nuisance_estimates['g'])
overlap_data_and_nuisance = data_and_nuisance_estimates[in_overlap_popluation] # only 428 units satisfy this
tau_hat, std_hat = psi_aiptw(**overlap_data_and_nuisance)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is 1613.301562438709 pm 1611.0472623428427


In [ ]:
# Finally, lets compare with the naive point estimate using only the Q model
# (to get a standard error, we'd need to bootstrap the model fitting)
Q1[treatment==1].mean() - Q0[treatment==1].mean()

1050.294554761258